In [1]:
import os
import pandas as pd

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

In [5]:
reader = Reader('HeatingCoilValve_ValidationWithB33Data.mat', "dymola")
df = pd.DataFrame()
df["time"] = list(reader.values("heaValSta_F.TSup")[0])
df["mea_tsup"] = list(reader.values("heaValSta_F.TSup")[1])
df["mea_tout"] = list(reader.values("heaValSta_F.TOut")[1])
df["tsup_set"] = list(reader.values("heaValSta_F.TSupSet")[1])
df["valve_trend"] = list(reader.values("percConvHeaValSig.y")[1])
df["cont_sig_modelica_alc"] = list(reader.values("heaValSta_F.yHeaVal")[1])

In [6]:
# list of even numbers range 0 - df.shape[0]
even_index = range(0, df.shape[0], 2)
df_at_change = df.loc[even_index, :].reset_index(drop = True)

In [55]:
def alc_controller(mea_T, set_T, int_error_prev_timestep, kp, ki):

    prop = kp * (set_T - mea_T)
    int_adder_this_timestep = ki * (set_T - (mea_T))
    inte = int_error_prev_timestep + int_adder_this_timestep
    
    sig = min(1, prop + inte)
    sig = max(sig, 0)
    
    return inte, sig, prop

In [56]:
df_at_change["inte"] = 0
df_at_change["sig"] = .5
df_at_change["prop"] = .5

In [57]:
for i in range(1, df_at_change.shape[0]):
    mea_t = df_at_change.loc[i, "mea_tsup"]
    set_t = df_at_change.loc[i, "tsup_set"]
    int_error_prev_timestep = df_at_change.loc[i - 1, "inte"]
    inte, sig, prop = alc_controller(mea_t, set_t, int_error_prev_timestep, .05, .005)

    df_at_change.loc[i, "sig"] = sig
    df_at_change.loc[i, "prop"] = prop
    df_at_change.loc[i, "inte"] = inte  

In [58]:
df_test = df_at_change.loc[1250:1300, :]
df_test

,time,mea_tsup,mea_tout,tsup_set,valve_trend,cont_sig_modelica_alc,inte,sig,prop
1250,7923827.0,65.300003,49.890003,66.0,0.508,1.0,0.256998,0.291998,0.035
1251,7923842.0,65.300003,49.890003,66.0,0.508,1.0,0.260498,0.295498,0.035
1252,7923857.0,65.300003,49.890003,66.0,0.508,1.0,0.263998,0.298998,0.035
1253,7923872.0,65.300003,49.890003,66.0,0.508,1.0,0.267498,0.302498,0.035
1254,7923887.0,65.300003,49.890003,66.0,0.508,1.0,0.270998,0.305998,0.035
1255,7923902.0,65.300003,49.890003,66.0,0.508,1.0,0.274498,0.309498,0.035
1256,7923917.0,65.300003,49.890003,66.0,0.508,1.0,0.277998,0.312998,0.035
1257,7923932.0,65.300003,49.890003,66.0,0.508,1.0,0.281498,0.316498,0.035
1258,7923947.0,65.300003,49.890003,66.0,0.508,1.0,0.284998,0.319998,0.035
1259,7923962.0,65.300003,49.890003,66.0,0.508,1.0,0.288498,0.323498,0.035


In [50]:
df_test.round(3).to_csv('test.csv')

In [28]:
round(66.5)

67.0